In [1]:
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt

In [2]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score
import gensim.models

In [45]:
df = pd.read_csv('data/data/usa_2012/ticker_data_preprocessed.csv', index_col=0)
print(df.shape)
df.head()

(435, 1197)


,2012-04-02,2012-04-03,2012-04-04,2012-04-05,2012-04-09,2012-04-10,2012-04-11,2012-04-12,2012-04-13,2012-04-16,...,2016-12-19,2016-12-20,2016-12-21,2016-12-22,2016-12-23,2016-12-27,2016-12-28,2016-12-29,2016-12-30,sector
A,0.008088,-0.000892,-0.024314,0.023548,-0.027250,-0.028703,0.004019,0.024959,-0.029635,-0.005919,...,-0.010455,0.017169,-0.003679,-0.001520,0.005003,0.006710,-0.016986,-0.001750,-0.001753,Healthcare
AAL,-0.015810,0.036145,0.015504,-0.010178,-0.003856,-0.036129,0.020080,0.035433,-0.006337,0.036990,...,0.019945,0.013174,0.005079,-0.015767,-0.004313,0.002682,-0.019338,-0.008811,-0.011852,Industrials
AAP,0.004629,0.011463,-0.003000,0.005238,-0.004213,-0.017145,0.015632,0.003234,0.002224,-0.004992,...,0.011110,0.010413,-0.012071,-0.017982,0.002934,0.005559,-0.008263,-0.000821,-0.006812,Consumer Cyclical
AAPL,0.031824,0.017280,-0.007961,0.015009,0.004024,-0.012244,-0.003564,-0.005478,-0.028164,-0.041472,...,0.005777,0.002658,0.000941,-0.006578,0.001978,0.006351,-0.004264,-0.000257,-0.007796,Technology
ABC,-0.011851,-0.013013,-0.005171,-0.006757,0.005756,-0.001301,0.010159,0.000774,-0.032981,-0.001332,...,0.013440,0.011222,-0.005675,-0.013190,0.009639,0.003310,-0.005963,0.012636,-0.014495,Healthcare


In [46]:
df_pct = df.drop(['sector'], axis=1).T
df_pct.index = pd.to_datetime(df_pct.index)

tickers_list = df_pct.columns.tolist()

df_pct.head()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADI,ADM,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
2012-04-02,0.008088,-0.015810,0.004629,0.031824,-0.011851,0.009013,-0.000816,0.015349,-0.012376,0.021162,...,0.010949,0.004805,0.004156,0.003920,0.007725,0.020541,-0.016578,0.015090,0.006557,-0.001398
2012-04-03,-0.000892,0.036145,0.011463,0.017280,-0.013013,-0.008933,-0.001633,-0.008246,-0.020551,-0.010826,...,-0.013989,0.001594,0.006772,-0.014241,-0.004946,-0.002472,0.010143,-0.001992,-0.006031,0.003733
2012-04-04,-0.024314,0.015504,-0.003000,-0.007961,-0.005171,-0.028842,0.005234,-0.007852,-0.009980,-0.017198,...,-0.011442,-0.015277,-0.004858,-0.009903,-0.016402,-0.015929,-0.009192,-0.008446,-0.014320,-0.019991
2012-04-05,0.023548,-0.010178,0.005238,0.015009,-0.006757,-0.002784,-0.002603,0.005431,0.002585,-0.003500,...,-0.004167,0.017615,-0.005633,-0.001883,0.003537,0.006115,0.008850,0.002323,-0.005910,0.000000
2012-04-09,-0.027250,-0.003856,-0.004213,0.004024,0.005756,-0.014891,-0.005383,-0.016206,-0.019335,-0.015326,...,-0.021850,-0.013657,-0.003399,-0.011082,-0.013092,-0.021809,-0.009621,-0.008807,-0.013872,-0.022770


In [47]:
scores = []
X = df_pct.values.reshape(-1,1)

In [48]:
X.shape

(520260, 1)

In [49]:
model = KMeans(n_clusters=200)
model.fit(X)
labels = model.predict(X)

In [50]:
X.shape, labels.shape

((520260, 1), (520260,))

In [51]:
mapping = {x:token for x, token in zip(X.flatten(), labels)}

In [52]:
df_pct_tok = df_pct.copy()
for col in df_pct:
    df_pct_tok[col] = df_pct[col].map(mapping)

In [53]:
df_pct_tok.head()

,A,AAL,AAP,AAPL,ABC,ABMD,ABT,ACN,ADI,ADM,...,WY,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION
2012-04-02,196,18,135,73,86,167,0,162,86,75,...,121,135,104,104,196,75,39,1,36,92
2012-04-03,131,136,26,51,4,84,92,149,154,32,...,64,183,119,64,9,88,40,38,161,104
2012-04-04,35,162,174,17,113,2,76,17,62,192,...,164,18,9,62,160,160,179,149,64,96
2012-04-05,147,159,76,1,53,88,88,76,199,127,...,193,163,63,38,191,36,167,95,63,117
2012-04-09,152,61,193,104,166,171,113,160,72,18,...,14,120,127,32,4,14,62,84,64,129


In [54]:
sentences = df_pct_tok.values.tolist()

In [55]:
model = gensim.models.Word2Vec(sentences, min_count=1)

In [56]:
embeddings = pd.DataFrame()
for col in df_pct_tok:
    embedding = np.zeros((100,))
    for i in df_pct_tok[col]:
        embedding += model.wv.get_vector(i)
    embedding /= df_pct_tok.shape[0]
    embeddings = pd.concat((embeddings, pd.Series(embedding)), axis=1)

In [57]:
embeddings

,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0,0.027565,0.084233,0.014973,0.027445,-0.011365,0.069233,-0.001709,-0.001617,0.013993,0.017029,...,0.018692,0.067669,-0.014854,-0.007957,0.000877,0.016008,0.011564,0.001474,0.031436,0.037232
1,0.032141,0.090952,0.016714,0.028802,0.012772,0.082311,0.010313,0.010974,0.021461,0.031093,...,0.024663,0.038703,0.007866,-0.008360,0.007297,0.023629,0.020552,0.015740,0.041307,0.056158
2,-0.113984,-0.134170,-0.109942,-0.115079,-0.094903,-0.126197,-0.101154,-0.101488,-0.108481,-0.109168,...,-0.110977,-0.131435,-0.096284,-0.100731,-0.104433,-0.109617,-0.107385,-0.102957,-0.113574,-0.115417
3,0.031708,-0.113313,0.066647,0.035482,0.109167,-0.081654,0.100620,0.097602,0.062638,0.048521,...,0.052019,-0.039281,0.121535,0.127470,0.098966,0.057521,0.067348,0.088661,0.015696,-0.009640
4,-0.008769,0.159147,-0.047586,-0.009883,-0.124400,0.114559,-0.098368,-0.097481,-0.048238,-0.038456,...,-0.034443,0.111025,-0.135156,-0.117935,-0.090256,-0.043577,-0.054912,-0.088452,0.002011,0.025617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.100569,0.113437,0.096698,0.100198,0.091225,0.109727,0.091986,0.093340,0.095821,0.097327,...,0.097520,0.105693,0.089911,0.089065,0.092855,0.097166,0.095844,0.093239,0.100680,0.103216
96,0.050965,0.063677,0.048493,0.050375,0.053507,0.064546,0.049894,0.049951,0.051743,0.055065,...,0.050698,0.046646,0.050918,0.044108,0.046786,0.051179,0.050617,0.051725,0.056064,0.060135
97,-0.018459,-0.073064,-0.005612,-0.018749,0.024780,-0.056997,0.012573,0.012153,-0.003715,-0.005343,...,-0.008937,-0.062743,0.027295,0.016712,0.008827,-0.005623,-0.001805,0.010002,-0.020384,-0.024860
98,-0.137874,-0.048097,-0.159432,-0.140530,-0.178695,-0.063997,-0.176904,-0.175790,-0.155171,-0.144157,...,-0.150049,-0.102599,-0.187061,-0.196785,-0.178229,-0.152435,-0.157686,-0.169551,-0.125779,-0.107933


In [58]:
embeddings.T.to_csv("signal2vec_embds2012.csv", index=None)